# Assignment 3
All questions are weighted the same in this assignment. This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. All questions are worth the same number of points except question 1 which is worth 17% of the assignment grade.

**Note**: Questions 3-13 rely on your question 1 answer.

In [1]:
import pandas as pd
import numpy as np

# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings
warnings.filterwarnings('ignore')

### Question 1
Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.  `'Switzerland17'` should be `'Switzerland'`.

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank".*

In [2]:
def answer_one():
    
    # ENERGY DATAFRAME
    
    # Reading df from an excel file
    Energy = pd.read_excel('assets/Energy Indicators.xls', sheet_name = 'Energy', skiprows = range(0, 17))
    
    # Dropping unecessary data
    Energy.drop(Energy.columns[[0,1]], axis=1, inplace=True)
    Energy.drop(Energy[Energy.index > 226].index, inplace=True)
    
    # Renaming headers
    headers = {'Unnamed: 2':'Country', 'Petajoules':'Energy Supply', 'Gigajoules':'Energy Supply per Capita', '%':'% Renewable'}
    Energy.rename(columns=headers, inplace=True)
    
    # Converting data to numeric and change the measure
    Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']] = Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].apply(pd.to_numeric, errors='coerce')
    Energy['Energy Supply'] = Energy['Energy Supply'] * 1000000
    
    # Formating Country names
    countries = {"Republic of Korea": "South Korea", "United States of America": "United States", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "China, Hong Kong Special Administrative Region": "Hong Kong"}
    Energy['Country'] = Energy['Country'].replace('\d+', '', regex=True)
    Energy['Country'] = Energy['Country'].replace('\(([^)]*)\)', '', regex=True)
    Energy['Country'] = Energy['Country'].replace(countries)
    Energy['Country'] = Energy['Country'].str.strip()
    
    # Setting 'Country' as index
    Energy.set_index('Country', inplace=True)
    
    
    #GDP DATAFRAME
    
    # Reading df from csv file
    GDP = pd.read_csv('assets/world_bank.csv', skiprows=4)
    
    # Formating Country names
    countries = {"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
    GDP['Country Name'] = GDP['Country Name'].replace(countries)
    
    # Setting 'Country' as index
    GDP.set_index('Country Name', inplace=True)
    
    
    # SCIMEN DATAFRAME
    
    # Reading  df from excel file
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
    
    # Setting 'Country' as index
    ScimEn.set_index('Country', inplace=True)
    
    # Sorting by Rank
    ScimEn.sort_values('Rank')
    
    
    # RESULT DATAFRAME
    
    # Merging ScimEn and Energy
    Result = pd.merge(ScimEn, Energy, how='left', left_index=True, right_index=True)
    
    # Filtering GDP in a new df
    GDP_filt = GDP.iloc[:, 49:].copy()
    
    # Merging Result and GDP_filt
    Result = pd.merge(Result, GDP_filt, how='left', left_index=True, right_index=True)
    
    return Result.iloc[0:15, :]
    
    raise NotImplementedError()

In [3]:
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"

assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"


### Question 2
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [4]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [5]:
def answer_two():
    
    # ENERGY DATAFRAME
    
    # Reading df from an excel file
    Energy = pd.read_excel('assets/Energy Indicators.xls', sheet_name = 'Energy', skiprows = range(0, 17))
    
    # Dropping unecessary data
    Energy.drop(Energy.columns[[0,1]], axis=1, inplace=True)
    Energy.drop(Energy[Energy.index > 226].index, inplace=True)
    
    # Renaming headers
    headers = {'Unnamed: 2':'Country', 'Petajoules':'Energy Supply', 'Gigajoules':'Energy Supply per Capita', '%':'% Renewable'}
    Energy.rename(columns=headers, inplace=True)
    
    # Converting data to numeric and change the measure
    Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']] = Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].apply(pd.to_numeric, errors='coerce')
    Energy['Energy Supply'] = Energy['Energy Supply'] * 1000000
    
    # Formating Country names
    countries = {"Republic of Korea": "South Korea", "United States of America": "United States", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "China, Hong Kong Special Administrative Region": "Hong Kong"}
    Energy['Country'] = Energy['Country'].replace('\d+', '', regex=True)
    Energy['Country'] = Energy['Country'].replace('\(([^)]*)\)', '', regex=True)
    Energy['Country'] = Energy['Country'].replace(countries)
    Energy['Country'] = Energy['Country'].str.strip()
    
    # Setting 'Country' as index
    Energy.set_index('Country', inplace=True)
    
    
    #GDP DATAFRAME
    
    # Reading df from csv file
    GDP = pd.read_csv('assets/world_bank.csv', skiprows=4)
    
    # Formating Country names
    countries = {"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
    GDP['Country Name'] = GDP['Country Name'].replace(countries)
    
    # Setting 'Country' as index
    GDP.set_index('Country Name', inplace=True)
    
    
    # SCIMEN DATAFRAME
    
    # Reading  df from excel file
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
    
    # Setting 'Country' as index
    ScimEn.set_index('Country', inplace=True)
    
    # Sorting by Rank
    ScimEn.sort_values('Rank')
    
    
    # RESULT DATAFRAME
    
    # Merging ScimEn and Energy
    Result_outer = pd.merge(ScimEn, Energy, how='outer', left_index=True, right_index=True)
    Result = pd.merge(ScimEn, Energy, how='left', left_index=True, right_index=True)
    
    # Merging Result and GDP_filt
    Result_outer = pd.merge(Result, GDP, how='outer', left_index=True, right_index=True)
    Result = pd.merge(Result, GDP, how='left', left_index=True, right_index=True)
    
    
    return len(Result_outer) - len(Result)
    
    raise NotImplementedError()

In [6]:
assert type(answer_two()) == int, "Q2: You should return an int number!"


### Question 3
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [7]:
def answer_three():
    
    # ENERGY DATAFRAME
    
    # Reading df from an excel file
    Energy = pd.read_excel('assets/Energy Indicators.xls', sheet_name = 'Energy', skiprows = range(0, 17))
    
    # Dropping unecessary data
    Energy.drop(Energy.columns[[0,1]], axis=1, inplace=True)
    Energy.drop(Energy[Energy.index > 226].index, inplace=True)
    
    # Renaming headers
    headers = {'Unnamed: 2':'Country', 'Petajoules':'Energy Supply', 'Gigajoules':'Energy Supply per Capita', '%':'% Renewable'}
    Energy.rename(columns=headers, inplace=True)
    
    # Converting data to numeric and change the measure
    Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']] = Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].apply(pd.to_numeric, errors='coerce')
    Energy['Energy Supply'] = Energy['Energy Supply'] * 1000000
    
    # Formating Country names
    countries = {"Republic of Korea": "South Korea", "United States of America": "United States", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "China, Hong Kong Special Administrative Region": "Hong Kong"}
    Energy['Country'] = Energy['Country'].replace('\d+', '', regex=True)
    Energy['Country'] = Energy['Country'].replace('\(([^)]*)\)', '', regex=True)
    Energy['Country'] = Energy['Country'].replace(countries)
    Energy['Country'] = Energy['Country'].str.strip()
    
    # Setting 'Country' as index
    Energy.set_index('Country', inplace=True)
    
    
    #GDP DATAFRAME
    
    # Reading df from csv file
    GDP = pd.read_csv('assets/world_bank.csv', skiprows=4)
    
    # Formating Country names
    countries = {"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
    GDP['Country Name'] = GDP['Country Name'].replace(countries)
    
    # Setting 'Country' as index
    GDP.set_index('Country Name', inplace=True)
    
    
    # SCIMEN DATAFRAME
    
    # Reading  df from excel file
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
    
    # Setting 'Country' as index
    ScimEn.set_index('Country', inplace=True)
    
    # Sorting by Rank
    ScimEn.sort_values('Rank')
    
    
    # RESULT DATAFRAME
    
    # Merging ScimEn and Energy
    Result = pd.merge(ScimEn, Energy, how='left', left_index=True, right_index=True)
    
    # Filtering GDP in a new df
    GDP_filt = GDP.iloc[:, 49:].copy()
    
    # Merging Result and GDP_filt
    Result = pd.merge(Result, GDP_filt, how='left', left_index=True, right_index=True)
    
    Result = Result[:15].copy()
    
    headers = GDP_filt.columns
    
    Result['avgGDP'] = Result.apply(lambda x: np.nanmean(x[headers]), axis=1)
    
    Result.sort_values('avgGDP', ascending=[0], inplace=True)
    
    avgGDP = pd.Series(Result['avgGDP'])
    
    return avgGDP
    
    
    raise NotImplementedError()

In [8]:
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"


### Question 4
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [9]:
def answer_four():    
    
    # ENERGY DATAFRAME
    
    # Reading df from an excel file
    Energy = pd.read_excel('assets/Energy Indicators.xls', sheet_name = 'Energy', skiprows = range(0, 17))
    
    # Dropping unecessary data
    Energy.drop(Energy.columns[[0,1]], axis=1, inplace=True)
    Energy.drop(Energy[Energy.index > 226].index, inplace=True)
    
    # Renaming headers
    headers = {'Unnamed: 2':'Country', 'Petajoules':'Energy Supply', 'Gigajoules':'Energy Supply per Capita', '%':'% Renewable'}
    Energy.rename(columns=headers, inplace=True)
    
    # Converting data to numeric and change the measure
    Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']] = Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].apply(pd.to_numeric, errors='coerce')
    Energy['Energy Supply'] = Energy['Energy Supply'] * 1000000
    
    # Formating Country names
    countries = {"Republic of Korea": "South Korea", "United States of America": "United States", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "China, Hong Kong Special Administrative Region": "Hong Kong"}
    Energy['Country'] = Energy['Country'].replace('\d+', '', regex=True)
    Energy['Country'] = Energy['Country'].replace('\(([^)]*)\)', '', regex=True)
    Energy['Country'] = Energy['Country'].replace(countries)
    Energy['Country'] = Energy['Country'].str.strip()
    
    # Setting 'Country' as index
    Energy.set_index('Country', inplace=True)
    
    
    #GDP DATAFRAME
    
    # Reading df from csv file
    GDP = pd.read_csv('assets/world_bank.csv', skiprows=4)
    
    # Formating Country names
    countries = {"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
    GDP['Country Name'] = GDP['Country Name'].replace(countries)
    
    # Setting 'Country' as index
    GDP.set_index('Country Name', inplace=True)
    
    
    # SCIMEN DATAFRAME
    
    # Reading  df from excel file
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
    
    # Setting 'Country' as index
    ScimEn.set_index('Country', inplace=True)
    
    # Sorting by Rank
    ScimEn.sort_values('Rank')
    
    
    # RESULT DATAFRAME
    
    # Merging ScimEn and Energy
    Result = pd.merge(ScimEn, Energy, how='left', left_index=True, right_index=True)
    
    # Filtering GDP in a new df
    GDP_filt = GDP.iloc[:, 49:].copy()
    
    # Merging Result and GDP_filt
    Result = pd.merge(Result, GDP_filt, how='left', left_index=True, right_index=True)
    
    
    # By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?
    headers = GDP_filt.columns
    
    Result['avgGDP'] = Result.apply(lambda x: np.nanmean(x[headers]), axis=1)
    
    Result.sort_values('avgGDP', ascending=[0], inplace=True)
    
    Change = Result.iloc[5, 19] - Result.iloc[5, 10]
    
    return Change
    
    
    raise NotImplementedError()

### Question 5
What is the mean energy supply per capita?

*This function should return a single number.*

In [11]:
def answer_five():
    
    # ENERGY DATAFRAME
    
    # Reading df from an excel file
    Energy = pd.read_excel('assets/Energy Indicators.xls', sheet_name = 'Energy', skiprows = range(0, 17))
    
    # Dropping unecessary data
    Energy.drop(Energy.columns[[0,1]], axis=1, inplace=True)
    Energy.drop(Energy[Energy.index > 226].index, inplace=True)
    
    # Renaming headers
    headers = {'Unnamed: 2':'Country', 'Petajoules':'Energy Supply', 'Gigajoules':'Energy Supply per Capita', '%':'% Renewable'}
    Energy.rename(columns=headers, inplace=True)
    
    # Converting data to numeric and change the measure
    Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']] = Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].apply(pd.to_numeric, errors='coerce')
    Energy['Energy Supply'] = Energy['Energy Supply'] * 1000000
    
    # Formating Country names
    countries = {"Republic of Korea": "South Korea", "United States of America": "United States", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "China, Hong Kong Special Administrative Region": "Hong Kong"}
    Energy['Country'] = Energy['Country'].replace('\d+', '', regex=True)
    Energy['Country'] = Energy['Country'].replace('\(([^)]*)\)', '', regex=True)
    Energy['Country'] = Energy['Country'].replace(countries)
    Energy['Country'] = Energy['Country'].str.strip()
    
    # Setting 'Country' as index
    Energy.set_index('Country', inplace=True)
    
    
    #GDP DATAFRAME
    
    # Reading df from csv file
    GDP = pd.read_csv('assets/world_bank.csv', skiprows=4)
    
    # Formating Country names
    countries = {"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
    GDP['Country Name'] = GDP['Country Name'].replace(countries)
    
    # Setting 'Country' as index
    GDP.set_index('Country Name', inplace=True)
    
    
    # SCIMEN DATAFRAME
    
    # Reading  df from excel file
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
    
    # Setting 'Country' as index
    ScimEn.set_index('Country', inplace=True)
    
    # Sorting by Rank
    ScimEn.sort_values('Rank')
    
    
    # RESULT DATAFRAME
    
    # Merging ScimEn and Energy
    Result = pd.merge(ScimEn, Energy, how='left', left_index=True, right_index=True)
    
    # Filtering GDP in a new df
    GDP_filt = GDP.iloc[:, 49:].copy()
    
    # Merging Result and GDP_filt
    Result = pd.merge(Result, GDP_filt, how='left', left_index=True, right_index=True)
    
    
    # What is the mean energy supply per capita?
    mean_energ_sup_per_cap = np.nanmean(Result['Energy Supply per Capita'][0:15])
    
    return mean_energ_sup_per_cap
    
    raise NotImplementedError()

### Question 6
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [12]:
def answer_six():
    
    
    # ENERGY DATAFRAME
    
    # Reading df from an excel file
    Energy = pd.read_excel('assets/Energy Indicators.xls', sheet_name = 'Energy', skiprows = range(0, 17))
    
    # Dropping unecessary data
    Energy.drop(Energy.columns[[0,1]], axis=1, inplace=True)
    Energy.drop(Energy[Energy.index > 226].index, inplace=True)
    
    # Renaming headers
    headers = {'Unnamed: 2':'Country', 'Petajoules':'Energy Supply', 'Gigajoules':'Energy Supply per Capita', '%':'% Renewable'}
    Energy.rename(columns=headers, inplace=True)
    
    # Converting data to numeric and change the measure
    Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']] = Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].apply(pd.to_numeric, errors='coerce')
    Energy['Energy Supply'] = Energy['Energy Supply'] * 1000000
    
    # Formating Country names
    countries = {"Republic of Korea": "South Korea", "United States of America": "United States", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "China, Hong Kong Special Administrative Region": "Hong Kong"}
    Energy['Country'] = Energy['Country'].replace('\d+', '', regex=True)
    Energy['Country'] = Energy['Country'].replace('\(([^)]*)\)', '', regex=True)
    Energy['Country'] = Energy['Country'].replace(countries)
    Energy['Country'] = Energy['Country'].str.strip()
    
    # Setting 'Country' as index
    Energy.set_index('Country', inplace=True)
    
    
    #GDP DATAFRAME
    
    # Reading df from csv file
    GDP = pd.read_csv('assets/world_bank.csv', skiprows=4)
    
    # Formating Country names
    countries = {"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
    GDP['Country Name'] = GDP['Country Name'].replace(countries)
    
    # Setting 'Country' as index
    GDP.set_index('Country Name', inplace=True)
    
    
    # SCIMEN DATAFRAME
    
    # Reading  df from excel file
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
    
    # Setting 'Country' as index
    ScimEn.set_index('Country', inplace=True)
    
    # Sorting by Rank
    ScimEn.sort_values('Rank')
    
    
    # RESULT DATAFRAME
    
    # Merging ScimEn and Energy
    Result = pd.merge(ScimEn, Energy, how='left', left_index=True, right_index=True)
    
    # Filtering GDP in a new df
    GDP_filt = GDP.iloc[:, 49:].copy()
    
    # Merging Result and GDP_filt
    Result = pd.merge(Result, GDP_filt, how='left', left_index=True, right_index=True)
    
    
    # What country has the maximum % Renewable and what is the percentage?
    max_renewable = pd.Series(Result['% Renewable'][:15])
    
    return (max_renewable.idxmax(), max_renewable.max())
    
    
    raise NotImplementedError()

In [13]:
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"

assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"


### Question 7
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [14]:
def answer_seven():
    
    # ENERGY DATAFRAME
    
    # Reading df from an excel file
    Energy = pd.read_excel('assets/Energy Indicators.xls', sheet_name = 'Energy', skiprows = range(0, 17))
    
    # Dropping unecessary data
    Energy.drop(Energy.columns[[0,1]], axis=1, inplace=True)
    Energy.drop(Energy[Energy.index > 226].index, inplace=True)
    
    # Renaming headers
    headers = {'Unnamed: 2':'Country', 'Petajoules':'Energy Supply', 'Gigajoules':'Energy Supply per Capita', '%':'% Renewable'}
    Energy.rename(columns=headers, inplace=True)
    
    # Converting data to numeric and change the measure
    Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']] = Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].apply(pd.to_numeric, errors='coerce')
    Energy['Energy Supply'] = Energy['Energy Supply'] * 1000000
    
    # Formating Country names
    countries = {"Republic of Korea": "South Korea", "United States of America": "United States", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "China, Hong Kong Special Administrative Region": "Hong Kong"}
    Energy['Country'] = Energy['Country'].replace('\d+', '', regex=True)
    Energy['Country'] = Energy['Country'].replace('\(([^)]*)\)', '', regex=True)
    Energy['Country'] = Energy['Country'].replace(countries)
    Energy['Country'] = Energy['Country'].str.strip()
    
    # Setting 'Country' as index
    Energy.set_index('Country', inplace=True)
    
    
    #GDP DATAFRAME
    
    # Reading df from csv file
    GDP = pd.read_csv('assets/world_bank.csv', skiprows=4)
    
    # Formating Country names
    countries = {"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
    GDP['Country Name'] = GDP['Country Name'].replace(countries)
    
    # Setting 'Country' as index
    GDP.set_index('Country Name', inplace=True)
    
    
    # SCIMEN DATAFRAME
    
    # Reading  df from excel file
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
    
    # Setting 'Country' as index
    ScimEn.set_index('Country', inplace=True)
    
    # Sorting by Rank
    ScimEn.sort_values('Rank')
    
    
    # RESULT DATAFRAME
    
    # Merging ScimEn and Energy
    Result = pd.merge(ScimEn, Energy, how='left', left_index=True, right_index=True)
    
    # Filtering GDP in a new df
    GDP_filt = GDP.iloc[:, 49:].copy()
    
    # Merging Result and GDP_filt
    Result = pd.merge(Result, GDP_filt, how='left', left_index=True, right_index=True)
    
    Result = Result[:15]
    
    # Create a new column that is the ratio of Self-Citations to Total Citations. 
    
    Result['Self_to_Total_Cit_Ratio'] = Result['Self-citations'] / Result['Citations']
    
    Ratio = pd.Series(Result['Self_to_Total_Cit_Ratio'])
    # What is the maximum value for this new column, and what country has the highest ratio?
    ratio_max_value = Ratio.max()
    ratio_max_country = Ratio.idxmax()
    
    return (ratio_max_country, ratio_max_value)
    
    
    raise NotImplementedError()

In [15]:
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"

assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"


### Question 8

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return the name of the country*

In [16]:
def answer_eight():
        
    
    # ENERGY DATAFRAME
    
    # Reading df from an excel file
    Energy = pd.read_excel('assets/Energy Indicators.xls', sheet_name = 'Energy', skiprows = range(0, 17))
    
    # Dropping unecessary data
    Energy.drop(Energy.columns[[0,1]], axis=1, inplace=True)
    Energy.drop(Energy[Energy.index > 226].index, inplace=True)
    
    # Renaming headers
    headers = {'Unnamed: 2':'Country', 'Petajoules':'Energy Supply', 'Gigajoules':'Energy Supply per Capita', '%':'% Renewable'}
    Energy.rename(columns=headers, inplace=True)
    
    # Converting data to numeric and change the measure
    Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']] = Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].apply(pd.to_numeric, errors='coerce')
    Energy['Energy Supply'] = Energy['Energy Supply'] * 1000000
    
    # Formating Country names
    countries = {"Republic of Korea": "South Korea", "United States of America": "United States", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "China, Hong Kong Special Administrative Region": "Hong Kong"}
    Energy['Country'] = Energy['Country'].replace('\d+', '', regex=True)
    Energy['Country'] = Energy['Country'].replace('\(([^)]*)\)', '', regex=True)
    Energy['Country'] = Energy['Country'].replace(countries)
    Energy['Country'] = Energy['Country'].str.strip()
    
    # Setting 'Country' as index
    Energy.set_index('Country', inplace=True)
    
    
    #GDP DATAFRAME
    
    # Reading df from csv file
    GDP = pd.read_csv('assets/world_bank.csv', skiprows=4)
    
    # Formating Country names
    countries = {"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
    GDP['Country Name'] = GDP['Country Name'].replace(countries)
    
    # Setting 'Country' as index
    GDP.set_index('Country Name', inplace=True)
    
    
    # SCIMEN DATAFRAME
    
    # Reading  df from excel file
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
    
    # Setting 'Country' as index
    ScimEn.set_index('Country', inplace=True)
    
    # Sorting by Rank
    ScimEn.sort_values('Rank')
    
    
    # RESULT DATAFRAME
    
    # Merging ScimEn and Energy
    Result = pd.merge(ScimEn, Energy, how='left', left_index=True, right_index=True)
    
    # Filtering GDP in a new df
    GDP_filt = GDP.iloc[:, 49:].copy()
    
    # Merging Result and GDP_filt
    Result = pd.merge(Result, GDP_filt, how='left', left_index=True, right_index=True)
    
    
    # Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
    Result['Population'] = Result['Energy Supply'] / Result['Energy Supply per Capita']
    
    
    # What is the third most populous country according to this estimate?
    Result.sort_values('Population', ascending=[0], inplace=True)
    third_pop = Result.iloc[2,]
    
    return third_pop.name
    
    
    raise NotImplementedError()

In [17]:
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"


### Question 9
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [18]:
def answer_nine():

    import scipy.stats as stats
    
    
    # ENERGY DATAFRAME
    
    # Reading df from an excel file
    Energy = pd.read_excel('assets/Energy Indicators.xls', sheet_name = 'Energy', skiprows = range(0, 17))
    
    # Dropping unecessary data
    Energy.drop(Energy.columns[[0,1]], axis=1, inplace=True)
    Energy.drop(Energy[Energy.index > 226].index, inplace=True)
    
    # Renaming headers
    headers = {'Unnamed: 2':'Country', 'Petajoules':'Energy Supply', 'Gigajoules':'Energy Supply per Capita', '%':'% Renewable'}
    Energy.rename(columns=headers, inplace=True)
    
    # Converting data to numeric and change the measure
    Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']] = Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].apply(pd.to_numeric, errors='coerce')
    Energy['Energy Supply'] = Energy['Energy Supply'] * 1000000
    
    # Formating Country names
    countries = {"Republic of Korea": "South Korea", "United States of America": "United States", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "China, Hong Kong Special Administrative Region": "Hong Kong"}
    Energy['Country'] = Energy['Country'].replace('\d+', '', regex=True)
    Energy['Country'] = Energy['Country'].replace('\(([^)]*)\)', '', regex=True)
    Energy['Country'] = Energy['Country'].replace(countries)
    Energy['Country'] = Energy['Country'].str.strip()
    
    # Setting 'Country' as index
    Energy.set_index('Country', inplace=True)
    
    
    #GDP DATAFRAME
    
    # Reading df from csv file
    GDP = pd.read_csv('assets/world_bank.csv', skiprows=4)
    
    # Formating Country names
    countries = {"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
    GDP['Country Name'] = GDP['Country Name'].replace(countries)
    
    # Setting 'Country' as index
    GDP.set_index('Country Name', inplace=True)
    
    
    # SCIMEN DATAFRAME
    
    # Reading  df from excel file
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
    
    # Setting 'Country' as index
    ScimEn.set_index('Country', inplace=True)
    
    # Sorting by Rank
    ScimEn.sort_values('Rank')
    
    
    # RESULT DATAFRAME
    
    # Merging ScimEn and Energy
    Result = pd.merge(ScimEn, Energy, how='left', left_index=True, right_index=True)
    
    # Filtering GDP in a new df
    GDP_filt = GDP.iloc[:, 49:].copy()
    
    # Merging Result and GDP_filt
    Result = pd.merge(Result, GDP_filt, how='left', left_index=True, right_index=True)
    
    
    # Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
    Result['Population'] = Result['Energy Supply'] / Result['Energy Supply per Capita']
    
    Result = Result[:15]
    
    # Create a column that estimates the number of citable documents per person. 
    Result['Cit_doc_per_person'] = Result['Citable documents'] / Result['Population']
    
    df = Result.loc[:, ['Cit_doc_per_person', 'Energy Supply per Capita']].copy()
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    
    # What is the correlation between the number of citable documents per capita and the energy supply per capita? 
    # Use the .corr() method, (Pearson's correlation).
    corr, pval=stats.pearsonr(df['Cit_doc_per_person'],df['Energy Supply per Capita'])
    
    
    return corr

    raise NotImplementedError()

In [19]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [20]:
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"


### Question 10
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [21]:
def answer_ten():
    
    
    # ENERGY DATAFRAME
    
    # Reading df from an excel file
    Energy = pd.read_excel('assets/Energy Indicators.xls', sheet_name = 'Energy', skiprows = range(0, 17))
    
    # Dropping unecessary data
    Energy.drop(Energy.columns[[0,1]], axis=1, inplace=True)
    Energy.drop(Energy[Energy.index > 226].index, inplace=True)
    
    # Renaming headers
    headers = {'Unnamed: 2':'Country', 'Petajoules':'Energy Supply', 'Gigajoules':'Energy Supply per Capita', '%':'% Renewable'}
    Energy.rename(columns=headers, inplace=True)
    
    # Converting data to numeric and change the measure
    Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']] = Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].apply(pd.to_numeric, errors='coerce')
    Energy['Energy Supply'] = Energy['Energy Supply'] * 1000000
    
    # Formating Country names
    countries = {"Republic of Korea": "South Korea", "United States of America": "United States", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "China, Hong Kong Special Administrative Region": "Hong Kong"}
    Energy['Country'] = Energy['Country'].replace('\d+', '', regex=True)
    Energy['Country'] = Energy['Country'].replace('\(([^)]*)\)', '', regex=True)
    Energy['Country'] = Energy['Country'].replace(countries)
    Energy['Country'] = Energy['Country'].str.strip()
    
    # Setting 'Country' as index
    Energy.set_index('Country', inplace=True)
    
    
    #GDP DATAFRAME
    
    # Reading df from csv file
    GDP = pd.read_csv('assets/world_bank.csv', skiprows=4)
    
    # Formating Country names
    countries = {"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
    GDP['Country Name'] = GDP['Country Name'].replace(countries)
    
    # Setting 'Country' as index
    GDP.set_index('Country Name', inplace=True)
    
    
    # SCIMEN DATAFRAME
    
    # Reading  df from excel file
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
    
    # Setting 'Country' as index
    ScimEn.set_index('Country', inplace=True)
    
    # Sorting by Rank
    ScimEn.sort_values('Rank')
    
    
    # RESULT DATAFRAME
    
    # Merging ScimEn and Energy
    Result = pd.merge(ScimEn, Energy, how='left', left_index=True, right_index=True)
    
    # Filtering GDP in a new df
    GDP_filt = GDP.iloc[:, 49:].copy()
    
    # Merging Result and GDP_filt
    Result = pd.merge(Result, GDP_filt, how='left', left_index=True, right_index=True)
    
    Result = Result[:15]
    
    # Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
    Result['Population'] = Result['Energy Supply'] / Result['Energy Supply per Capita']
    
    
    # Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, 
    # and a 0 if the country's % Renewable value is below the median.
    
    renew_value_avg = np.median(Result.iloc[:15,9])
    
    Result['HighRenew'] = Result['% Renewable'].apply(lambda x: 1 if (x >= renew_value_avg) else 0)
    
    Result.sort_values('Rank',ascending=True, inplace=True)
    
    return Result['HighRenew']

    raise NotImplementedError()

In [22]:
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"


### Question 11
Use the following dictionary to group the Countries by Continent, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [23]:
def answer_eleven():
    
    # ENERGY DATAFRAME
    
    # Reading df from an excel file
    Energy = pd.read_excel('assets/Energy Indicators.xls', sheet_name = 'Energy', skiprows = range(0, 17))
    
    # Dropping unecessary data
    Energy.drop(Energy.columns[[0,1]], axis=1, inplace=True)
    Energy.drop(Energy[Energy.index > 226].index, inplace=True)
    
    # Renaming headers
    headers = {'Unnamed: 2':'Country', 'Petajoules':'Energy Supply', 'Gigajoules':'Energy Supply per Capita', '%':'% Renewable'}
    Energy.rename(columns=headers, inplace=True)
    
    # Converting data to numeric and change the measure
    Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']] = Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].apply(pd.to_numeric, errors='coerce')
    Energy['Energy Supply'] = Energy['Energy Supply'] * 1000000
    
    # Formating Country names
    countries = {"Republic of Korea": "South Korea", "United States of America": "United States", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "China, Hong Kong Special Administrative Region": "Hong Kong"}
    Energy['Country'] = Energy['Country'].replace('\d+', '', regex=True)
    Energy['Country'] = Energy['Country'].replace('\(([^)]*)\)', '', regex=True)
    Energy['Country'] = Energy['Country'].replace(countries)
    Energy['Country'] = Energy['Country'].str.strip()
    
    # Setting 'Country' as index
    Energy.set_index('Country', inplace=True)
    
    
    #GDP DATAFRAME
    
    # Reading df from csv file
    GDP = pd.read_csv('assets/world_bank.csv', skiprows=4)
    
    # Formating Country names
    countries = {"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
    GDP['Country Name'] = GDP['Country Name'].replace(countries)
    
    # Setting 'Country' as index
    GDP.set_index('Country Name', inplace=True)
    
    
    # SCIMEN DATAFRAME
    
    # Reading  df from excel file
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
    
    # Setting 'Country' as index
    ScimEn.set_index('Country', inplace=True)
    
    # Sorting by Rank
    ScimEn.sort_values('Rank')
    
    
    # RESULT DATAFRAME
    
    # Merging ScimEn and Energy
    Result = pd.merge(ScimEn, Energy, how='left', left_index=True, right_index=True)
    
    # Filtering GDP in a new df
    GDP_filt = GDP.iloc[:, 49:].copy()
    
    # Merging Result and GDP_filt
    Result = pd.merge(Result, GDP_filt, how='left', left_index=True, right_index=True)
    
    
    # Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
    Result['Population'] = Result['Energy Supply'] / Result['Energy Supply per Capita']
    
    
    # Use the following dictionary to group the Countries by Continent, 
    # then create a DataFrame that displays the sample size (the number of countries in each continent bin), 
    # and the sum, mean, and std deviation for the estimated population of each country.
    
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    df = pd.DataFrame(ContinentDict.items(), columns=['Country', 'Continent'])
    df.set_index('Country', inplace=True)
    
    df = pd.merge(df, Result['Population'], how='left', left_index=True, right_index=True)
    
    df2 = df.groupby('Continent').Population.agg(['count', 'sum', 'mean', 'std'])
    df2.rename(columns={'count':'size'}, inplace=True)
    
    return df2

    raise NotImplementedError()

In [25]:
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"

assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"

assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"


### Question 12
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [26]:
def answer_twelve():
        
    # ENERGY DATAFRAME
    
    # Reading df from an excel file
    Energy = pd.read_excel('assets/Energy Indicators.xls', sheet_name = 'Energy', skiprows = range(0, 17))
    
    # Dropping unecessary data
    Energy.drop(Energy.columns[[0,1]], axis=1, inplace=True)
    Energy.drop(Energy[Energy.index > 226].index, inplace=True)
    
    # Renaming headers
    headers = {'Unnamed: 2':'Country', 'Petajoules':'Energy Supply', 'Gigajoules':'Energy Supply per Capita', '%':'% Renewable'}
    Energy.rename(columns=headers, inplace=True)
    
    # Converting data to numeric and change the measure
    Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']] = Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].apply(pd.to_numeric, errors='coerce')
    Energy['Energy Supply'] = Energy['Energy Supply'] * 1000000
    
    # Formating Country names
    countries = {"Republic of Korea": "South Korea", "United States of America": "United States", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "China, Hong Kong Special Administrative Region": "Hong Kong"}
    Energy['Country'] = Energy['Country'].replace('\d+', '', regex=True)
    Energy['Country'] = Energy['Country'].replace('\(([^)]*)\)', '', regex=True)
    Energy['Country'] = Energy['Country'].replace(countries)
    Energy['Country'] = Energy['Country'].str.strip()
    
    # Setting 'Country' as index
    Energy.set_index('Country', inplace=True)
    
    
    #GDP DATAFRAME
    
    # Reading df from csv file
    GDP = pd.read_csv('assets/world_bank.csv', skiprows=4)
    
    # Formating Country names
    countries = {"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
    GDP['Country Name'] = GDP['Country Name'].replace(countries)
    
    # Setting 'Country' as index
    GDP.set_index('Country Name', inplace=True)
    
    
    # SCIMEN DATAFRAME
    
    # Reading  df from excel file
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
    
    # Setting 'Country' as index
    ScimEn.set_index('Country', inplace=True)
    
    # Sorting by Rank
    ScimEn.sort_values('Rank')
    
    
    # RESULT DATAFRAME
    
    # Merging ScimEn and Energy
    Result = pd.merge(ScimEn, Energy, how='left', left_index=True, right_index=True)
    
    # Filtering GDP in a new df
    GDP_filt = GDP.iloc[:, 49:].copy()
    
    # Merging Result and GDP_filt
    Result = pd.merge(Result, GDP_filt, how='left', left_index=True, right_index=True)
    
    
    # Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
    Result['Population'] = Result['Energy Supply'] / Result['Energy Supply per Capita']
    
    
    # Use the following dictionary to group the Countries by Continent, 
    # then create a DataFrame that displays the sample size (the number of countries in each continent bin), 
    # and the sum, mean, and std deviation for the estimated population of each country.
    
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
   
    # Appending Continents to dataframe
    df = pd.DataFrame(ContinentDict.items(), columns=['Country', 'Continent'])
    df.set_index('Country', inplace=True)
    
    Result = pd.merge(Result, df, how='left', left_index=True, right_index=True)
    
    
   # Selecting top15
    top15 = Result.iloc[:15,:].copy()
    
    
    top15.rename(columns={'% Renewable':'% Renewable values'}, inplace=True)
    
    
    # Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. 
    top15['% Renewable'] = pd.cut(top15['% Renewable values'],5)
    
    top15.set_index(['Continent', '% Renewable'], inplace=True)
    
    
    # How many countries are in each of these groups?
    top15 = top15.groupby(['Continent', '% Renewable'])['% Renewable values'].agg('count')
    
    return top15[top15 > 0]


    raise NotImplementedError()


In [27]:
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"

assert len(answer_twelve()) == 9, "Q12: Wrong result numbers!"


### Question 13
Convert the Population Estimate series to a string with thousands separator (using commas). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12,345,678.90

*This function should return a series `PopEst` whose index is the country name and whose values are the population estimate string*

In [28]:
def answer_thirteen():
    
        # ENERGY DATAFRAME
    
    # Reading df from an excel file
    Energy = pd.read_excel('assets/Energy Indicators.xls', sheet_name = 'Energy', skiprows = range(0, 17))
    
    # Dropping unecessary data
    Energy.drop(Energy.columns[[0,1]], axis=1, inplace=True)
    Energy.drop(Energy[Energy.index > 226].index, inplace=True)
    
    # Renaming headers
    headers = {'Unnamed: 2':'Country', 'Petajoules':'Energy Supply', 'Gigajoules':'Energy Supply per Capita', '%':'% Renewable'}
    Energy.rename(columns=headers, inplace=True)
    
    # Converting data to numeric and change the measure
    Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']] = Energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].apply(pd.to_numeric, errors='coerce')
    Energy['Energy Supply'] = Energy['Energy Supply'] * 1000000
    
    # Formating Country names
    countries = {"Republic of Korea": "South Korea", "United States of America": "United States", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "China, Hong Kong Special Administrative Region": "Hong Kong"}
    Energy['Country'] = Energy['Country'].replace('\d+', '', regex=True)
    Energy['Country'] = Energy['Country'].replace('\(([^)]*)\)', '', regex=True)
    Energy['Country'] = Energy['Country'].replace(countries)
    Energy['Country'] = Energy['Country'].str.strip()
    
    # Setting 'Country' as index
    Energy.set_index('Country', inplace=True)
    
    
    #GDP DATAFRAME
    
    # Reading df from csv file
    GDP = pd.read_csv('assets/world_bank.csv', skiprows=4)
    
    # Formating Country names
    countries = {"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
    GDP['Country Name'] = GDP['Country Name'].replace(countries)
    
    # Setting 'Country' as index
    GDP.set_index('Country Name', inplace=True)
    
    
    # SCIMEN DATAFRAME
    
    # Reading  df from excel file
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
    
    # Setting 'Country' as index
    ScimEn.set_index('Country', inplace=True)
    
    # Sorting by Rank
    ScimEn.sort_values('Rank')
    
    
    # RESULT DATAFRAME
    
    # Merging ScimEn and Energy
    Result = pd.merge(ScimEn, Energy, how='left', left_index=True, right_index=True)
    
    # Filtering GDP in a new df
    GDP_filt = GDP.iloc[:, 49:].copy()
    
    # Merging Result and GDP_filt
    Result = pd.merge(Result, GDP_filt, how='left', left_index=True, right_index=True)
    
    
    # Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
    Result['Population'] = Result['Energy Supply'] / Result['Energy Supply per Capita']
    Result['Population'] = Result['Population'].apply(lambda x: "{:,}".format(x))
    Result['Population'] = Result['Population'].apply(str)
    return Result['Population'][:15]
    
    raise NotImplementedError()

In [29]:
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"

assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"


### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [30]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")